<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment3_LinearModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries that will be used 
import numpy as np
import tarfile
import glob
import re
import pandas as pd
#from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Downloading the dataset

In [ ]:
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz

--2021-10-01 04:37:53--  http://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4029756 (3.8M) [application/x-gzip]
Saving to: ‘scale_data.tar.gz’

scale_data.tar.gz   100%[===================>]   3.84M  5.40MB/s    in 0.7s    

2021-10-01 04:37:54 (5.40 MB/s) - ‘scale_data.tar.gz’ saved [4029756/4029756]



In [ ]:
# Untar the dataset
my_tar = tarfile.open('/content/scale_data.tar.gz')
my_tar.extractall('/content/') 
my_tar.close()


In [ ]:
for path in glob.glob('/content/scaledata/*/*'):
    fo = open(path)
    doc = fo.read()
    print(path,'\n')
    print(doc[:10])

/content/scaledata/Scott+Renshaw/label.3class.Scott+Renshaw 

0
0
0
0
0

/content/scaledata/Scott+Renshaw/label.4class.Scott+Renshaw 

0
0
0
0
0

/content/scaledata/Scott+Renshaw/rating.Scott+Renshaw 

0
0
0
0
0.
/content/scaledata/Scott+Renshaw/subj.Scott+Renshaw 

i'm guessi
/content/scaledata/Scott+Renshaw/id.Scott+Renshaw 

11961
1391
/content/scaledata/Dennis+Schwartz/subj.Dennis+Schwartz 

in my opin
/content/scaledata/Dennis+Schwartz/label.4class.Dennis+Schwartz 

0
0
0
0
0

/content/scaledata/Dennis+Schwartz/rating.Dennis+Schwartz 

0.1
0.2
0.
/content/scaledata/Dennis+Schwartz/label.3class.Dennis+Schwartz 

0
0
0
0
0

/content/scaledata/Dennis+Schwartz/id.Dennis+Schwartz 

29420
1721
/content/scaledata/Steve+Rhodes/subj.Steve+Rhodes 

this bit o
/content/scaledata/Steve+Rhodes/id.Steve+Rhodes 

11790
1762
/content/scaledata/Steve+Rhodes/label.4class.Steve+Rhodes 

0
0
0
0
0

/content/scaledata/Steve+Rhodes/rating.Steve+Rhodes 

0.1
0.1
0.
/content/scaledata/Steve+Rhodes/label.